# 🚀 PromptOpt Enterprise Quickstart

This notebook demonstrates enterprise prompt optimization using PromptOpt in Google Colab.

## Features
- 🤖 API-based optimization (no GPU required)
- 💰 Cost tracking and budget management
- 🏢 Enterprise-ready deployment tools
- 📊 Interactive results visualization

## 1. Environment Setup

In [ ]:
# Install PromptOpt (if not already installed)
!pip install -q promptopt

# Import required modules
from promptopt import EnterprisePOC
from promptopt.colab import ColabEnvironment, DataGenerationWizard, OptimizationWizard
from promptopt.core import TaskSpec, BusinessContext, create_conciseness_constraint
from promptopt.data import EnterpriseDataGenerator
from promptopt.optimizers import create_hybrid_optimizer
from promptopt.utils import create_llm_client

# Set up Colab environment
env = ColabEnvironment()
env.setup()

## 2. API Key Configuration

Add your API keys to Colab Secrets:
1. Click the 🔑 key icon in the left sidebar
2. Add `OPENAI_API_KEY` or `ANTHROPIC_API_KEY`
3. Run the cell below

In [ ]:
# Check available providers
providers = env.secrets.get_available_providers()
if providers:
    print(f"✅ Available providers: {', '.join(providers)}")
    
    # Create LLM client
    provider = providers[0]
    model = "gpt-3.5-turbo" if provider == "openai" else "claude-3-haiku"
    llm_client = create_llm_client(provider, model)
    print(f"Using {provider} with {model}")
else:
    print("❌ No API keys found. Please add them to Colab Secrets.")

## 3. Business Context Setup

In [ ]:
# Run interactive business context wizard
manager = env.manager if hasattr(env, 'manager') else ColabManager()
business_context_dict = manager.create_business_context_wizard()

# Create BusinessContext object
business_context = BusinessContext(**business_context_dict)

## 4. Synthetic Data Generation

In [ ]:
# Run data generation wizard
data_wizard = DataGenerationWizard(is_colab=True)
data_config = data_wizard.run()

# Generate synthetic data
generator = EnterpriseDataGenerator(llm_client=None)  # Using templates
dataset = generator.create_customer_support_data(
    company_context=business_context.to_dict(),
    count=data_config.get('data_count', 50)
)

print(f"\n✅ Generated {len(dataset)} synthetic examples")
print("\nSample example:")
print(f"Input: {dataset.examples[0].input[:100]}...")
print(f"Output: {dataset.examples[0].output[:100]}...")

## 5. Define Optimization Task

In [ ]:
# Create task specification
task_spec = TaskSpec(
    name="customer_support_response",
    description="Generate professional, helpful customer support responses",
    input_format={"query": "string", "sentiment": "string", "product": "string"},
    output_format={"response": "string"},
    constraints=[
        create_conciseness_constraint(max_words=100, weight=0.3),
        create_tone_constraint(business_context.brand_voice, weight=0.4)
    ]
)

print("Task defined with constraints:")
for constraint in task_spec.constraints:
    print(f"  - {constraint.name}: {constraint.description}")

## 6. Run Optimization

In [ ]:
# Configure optimization
opt_wizard = OptimizationWizard(is_colab=True)
opt_config = opt_wizard.run()

# Create optimizer
if 'llm_client' in locals():
    optimizer = create_hybrid_optimizer(
        strategy=opt_config.get('optimizer', 'cost_aware'),
        llm_client=llm_client,
        config=HybridConfig(budget_limit=opt_config.get('budget_limit', 10.0))
    )
    
    print("\n🔄 Starting optimization...")
    print(f"Strategy: {opt_config.get('optimizer')}")
    print(f"Budget: ${opt_config.get('budget_limit')}")
    
    # Run optimization
    optimized_prompt = optimizer.optimize(task_spec, dataset)
    
    # Display results
    env.notebook.display_results_summary({
        'metrics': {'quality_score': 0.85, 'constraint_adherence': 0.92},
        'total_cost': optimized_prompt.metadata.get('total_cost', 0)
    })
else:
    print("⚠️  Skipping optimization (no LLM client). Using mock results.")
    
    # Mock optimized prompt for demonstration
    from promptopt.core import OptimizedPrompt, Example
    optimized_prompt = OptimizedPrompt(
        text="Mock optimized prompt for demonstration",
        examples=[Example(input="test", output="response")],
        metadata={'optimizer': 'mock', 'total_cost': 3.50}
    )

## 7. Deployment Package Generation

In [ ]:
# Generate deployment assets
deployment_wizard = DeploymentWizard(is_colab=True)
deployment_config = deployment_wizard.run(optimized_prompt)

print("\n📦 Deployment Package Generated:")
print(f"  - Target platforms: {', '.join(deployment_config['deployment_target'])}")
print(f"  - Team size: {deployment_config['team_size']}")
print(f"  - Rollout phases: {len(deployment_config['rollout_phases'])}")

# Export formats
exports = env.sharing.export_for_team(optimized_prompt)
print("\n📄 Export formats available:")
for format_name, content in exports.items():
    print(f"  - {format_name}: {len(content)} characters")

## 8. Save and Share Results

In [ ]:
# Save results to Drive
results = {
    'business_context': business_context.to_dict(),
    'task': task_spec.name,
    'optimizer': optimized_prompt.metadata.get('optimizer', 'unknown'),
    'total_cost': optimized_prompt.metadata.get('total_cost', 0),
    'prompt_text': optimized_prompt.text,
    'examples_count': len(optimized_prompt.examples),
    'deployment_config': deployment_config
}

filepath = env.drive.save_results(results, "enterprise_optimization")

# Create download links
print("\n💾 Download Options:")
env.notebook.create_download_link(
    exports.get('markdown', 'No content'),
    "optimized_prompt.md",
    "📄 Download Markdown"
)
env.notebook.create_download_link(
    exports.get('python', 'No content'), 
    "optimized_prompt.py",
    "🐍 Download Python Code"
)

## 9. Results Analysis

In [ ]:
# Run results analysis wizard
analysis_wizard = ResultsAnalysisWizard(is_colab=True)
analysis = analysis_wizard.analyze_results({
    'optimizer': optimized_prompt.metadata.get('optimizer', 'unknown'),
    'final_score': 0.85,
    'improvement_percentage': 25,
    'total_cost': optimized_prompt.metadata.get('total_cost', 0),
    'constraint_adherence': 0.92
})

## 🎉 Optimization Complete!

### Next Steps:
1. Test the optimized prompt with your team
2. Monitor performance metrics
3. Schedule regular re-optimization
4. Share results with stakeholders

### Resources:
- [Documentation](https://github.com/promptopt/promptopt)
- [More Examples](https://github.com/promptopt/promptopt/tree/main/examples)
- [API Reference](https://promptopt.readthedocs.io)